In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

# Create 5 multi-server queues with PS scheduling
node = np.empty(5, dtype=object)
node[0] = Queue(model, 'Queue1', SchedStrategy.PS)
node[0].set_number_of_servers(1)
node[1] = Queue(model, 'Queue2', SchedStrategy.PS)
node[1].set_number_of_servers(2)
node[2] = Queue(model, 'Queue3', SchedStrategy.PS)
node[2].set_number_of_servers(3)
node[3] = Queue(model, 'Queue4', SchedStrategy.PS)  # only closed classes
node[3].set_number_of_servers(4)
node[4] = Queue(model, 'Queue5', SchedStrategy.PS)  # only open classes
node[4].set_number_of_servers(5)

source = Source(model, 'Source')
sink = Sink(model, 'Sink')

# Create job classes
jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'ClosedClass', 3, node[0], 0)
jobclass[1] = OpenClass(model, 'OpenClass', 0)

# Set service rates
for i in range(5):
    node[i].set_service(jobclass[0], Exp(i+1))
    node[i].set_service(jobclass[1], Exp(np.sqrt(i+1)))

# Set arrival rate for open class
source.set_arrival(jobclass[1], Exp(0.3))

In [3]:
# Setup routing matrix
M = model.get_number_of_stations()
K = model.get_number_of_classes()

P = model.init_routing_matrix()

# Closed class routing: Queue1 -> Queue2 -> Queue3 -> Queue4 -> Queue1 (circular)
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)
P.set(jobclass[0], jobclass[0], node[1], node[2], 1.0)
P.set(jobclass[0], jobclass[0], node[2], node[3], 1.0)
P.set(jobclass[0], jobclass[0], node[3], node[0], 1.0)

# Open class routing: Source -> Queue1 -> Queue2 -> Queue3 -> Queue5 -> Sink
P.set(jobclass[1], jobclass[1], source, node[0], 1.0)
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[2], 1.0)
P.set(jobclass[1], jobclass[1], node[2], node[4], 1.0)
P.set(jobclass[1], jobclass[1], node[4], sink, 1.0)

model.link(P)

In [ ]:
print('This example shows the execution of the solver on a 2-class mixed model with 5 multi-server nodes.')

# Create solvers
solver = np.array([], dtype=object)
solver = np.append(solver, CTMC(model, cutoff=3))  # CTMC is infinite on this model
solver = np.append(solver, JMT(model, samples=100000, seed=23000))
solver = np.append(solver, MVA(model, method='exact'))
# Note: MAM does not support mixed models with both open and closed classes
solver = np.append(solver, DES(model, samples=100000, seed=23000))

# Solve and display results
AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].get_name()}')
    AvgTable[s] = solver[s].avg_table()
    print(AvgTable[s])

In [ ]:
# Network topology visualization is not available in Python implementation
print("Network created with 5 multi-server PS queues")
print("  - Queue1: 1 server")
print("  - Queue2: 2 servers")
print("  - Queue3: 3 servers")
print("  - Queue4: 4 servers (closed class only)")
print("  - Queue5: 5 servers (open class only)")